# Introduction to NMR processing

## Library Imports

In [1]:
from spectrochempy import *
import os

SpectroChemPy's API - v.0.1a3.dev24+gab598c30.d20181226
© Copyright 2014-2018 - A.Travert & C.Fernandez @ LCS


## Import data

Here we import two dataset, one is 1D and the other is 2D

Because , we will sometimes need to recall the original dataset, we create to getting functions

In [2]:
# 1D dataset getting function 
def get_dataset1D():
    dataset1D = NDDataset()
    path = os.path.join(datadir.path, 'nmrdata','bruker', 'tests', 'nmr','bruker_1d')
    dataset1D.read_bruker_nmr(path, expno=1, remove_digital_filter=True)
    return dataset1D

# 2D dataset getting function
def get_dataset2D():
    dataset2D = NDDataset()
    path = os.path.join(datadir.path, 'nmrdata','bruker', 'tests', 'nmr','bruker_2d')
    dataset2D.read_bruker_nmr(path, expno=1, remove_digital_filter=True)
    return dataset2D

In [3]:
dataset1D = get_dataset1D()
dataset1D # display info

NDDataset: [-1037.267, -1077.841, ...,   -0.053,    0.101] unitless

In [4]:
dataset2D = get_dataset2D()
dataset2D

NDDataset: [[   0.062,    0.224, ...,    0.031,   -0.033],
            [  -0.003,    0.162, ...,   -0.081,    0.011],
            ...,
            [   0.000,   -0.000, ...,    0.000,   -0.000],
            [   0.000,   -0.000, ...,    0.000,   -0.000]] unitless

## Plot the 1D dataset raw data

In [5]:
# plot the real data
dataset1D.plot(xlim=(0,25000)) 
   
# plot the imaginary data on the same plot
dataset1D.plot(imag=True, data_only=True, clear=False)
    # `clear=False`to plot on the previous plot
    # `data_only=True` to plot only the additional data, without updating the figure setting 
    # such as xlim and so on.

FigureCanvasNbAgg()

True

To display the imaginary part, one can also simply use the show_complex commands.

In [6]:
dataset1D.plot(show_complex=True, color='green',
                xlim=(0.,20000.))

FigureCanvasNbAgg()

## Plot the 2D dataset raw data

In [7]:
dataset2D = get_dataset2D()
dataset2D.plot_map(xlim=(0.,25000.))

FigureCanvasNbAgg()

probably less util, but multiple display is also possible

In [8]:
dataset2D.plot_map(xlim=(0.,6000.))
dataset2D.T.plot_map(cmap='magma', xlim=(0.,6000.), data_only=True, clear=False)

FigureCanvasNbAgg()

## Apodization

In [9]:
# Plot
dataset1D = get_dataset1D() # restore original
dataset1D.plot() 

# Create the apodized dataset
lb_dataset = dataset1D.em(lb=100.*ur.Hz)

lb_dataset.plot(xlim=(0,25000), zlim=(-100,100), color='r', clear=False)

t = lb_dataset.ax.text(12500,90,'Dual display (original & apodized fids)', ha='center', fontsize=10)

FigureCanvasNbAgg()

In [10]:
# Plot
dataset1D = get_dataset1D() # restore original

# Create the apodized dataset
lb_dataset = em(dataset1D, lb=100.*ur.Hz, inplace=False)

dataset1D.plot() 
lb_dataset.plot(xlim=(0,25000), zlim=(-100,100), color='r', clear=False)
t = lb_dataset.ax.text(12500,90,'Dual display (original & apodized fids)', ha='center', fontsize=10)

FigureCanvasNbAgg()

Note that the apodized dataset actually replace the original data

In [11]:
# check that both dataset are the same
lb_dataset is dataset1D  # note here, that the original data are modified by default 
                       # when applying apodization function. 
                       # Use the `inplace` keyword to modify this behavior

False

If we want to avoid this behavior and create a new dataset instead, we use the `inplace` flag.

In [12]:
dataset1D = get_dataset1D()

lb2_dataset = dataset1D.em(lb=100.*ur.Hz, inplace=False)

# check that both dataset are different
lb2_dataset is not dataset1D

True

We can also get only the apodization function

In [13]:
# Plot
dataset1D = get_dataset1D() # restore original
dataset1D.plot() 

# create the apodized dataset (if apply is False, the apodization function is not applied to the dataset, 
# but returned)
apodfunc = dataset1D.em(lb=100.*ur.Hz, apply=False)*200

apodfunc.plot(color='r', clear=False)

dataset1D.em(lb=100.*ur.Hz, apply=True)
dataset1D.plot(data_only=True, xlim=(0,25000), zlim=(-200,200), color='g', clear=False) 

t = dataset1D.ax.text(12500,180,'Multiple display (original & em apodized fids + apod.function)', ha='center', fontsize=10)

FigureCanvasNbAgg()

Apodization function can be em, gm, sp ...

In [14]:
# Plot
dataset1D = get_dataset1D() # restore original
# normalize amplitude
dataset1D /= dataset1D.data.max()

dataset1D.plot() 

LB = 100.*ur.Hz
GB = 300.*ur.Hz
apodfunc = dataset1D.gm(gb=GB, lb=LB, apply=False)

apodfunc.plot(color='r', clear=False)

dataset1D.gm(gb=GB, lb=LB) #  apply=True by default

dataset1D.plot(xlim=(0,25000), zlim=(-1,1), color='g', clear=False) 

t = dataset1D.ax.text(12500,.70,'Multiple display (original & gm apodized fids + apod.function)', ha='center', fontsize=10)

FigureCanvasNbAgg()

**TODO**: sp function

In [20]:
# Plot
dataset1D = get_dataset1D() # restore original
# normalize amplitude
dataset1D /= dataset1D.data.max()

dataset1D.plot() 

apodfunc = dataset1D.sp(off=0, end=10., pow=1.2, apply=False)

apodfunc.plot(color='r', clear=False)

FigureCanvasNbAgg()

[SpectroChemPy] ERROR | TypeError: 'NoneType' object is not callable


### Apodization of 2D data

In [16]:
dataset2D = get_dataset2D()
dataset2D.plot_map(xlim=(0.,25000.))

LB = 200.*ur.Hz
dataset2D.em(lb=LB)
dataset2D.em(lb=LB/2, axis=0)  
dataset2D.plot_map(data_only=True, cmap='copper', clear=False)

FigureCanvasNbAgg()

## Time-frequency transforms : FFT

In [17]:
dataset1D = get_dataset1D() # restore original
LB = 10.*ur.Hz
dataset1D.em(lb=LB)
#dataset1D.zf_auto(inplace=True)
#transf1 = source1D.fft() # by defauut fft create a new dataset

NDDataset: [-1037.267, -1077.841, ...,   -0.011,    0.021] unitless

In [18]:
dataset1D = get_dataset1D() # restore original
LB = 10.*ur.Hz
GB = 50.*ur.Hz
dataset1D.gm(gb=GB, lb=LB)
#dataset1D.zf_auto()
#transf2 = dataset1D.fft()

NDDataset: [-1037.267, -1077.841, ...,   -0.000,    0.000] unitless

As the new dataset is transformed, function that apply to time data such as **em** should not work

In [19]:
#_ = transf1.em(lb=10*ur.Hz)